In [1]:
import os
import pandas as pd
import numpy as np
from IPython import get_ipython

# envs
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
os.getcwd()

'/home/runner/work/update/update'

In [3]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting to check: ", notebook_path)
    


    # if there is no schedule, run immediately
    if schedule_in_days == None:
        EXECUTE = True
        


    # else run on a schedule based on last execution time
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = ["2002-07-16 18:32",np.nan]
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        import datetime
        now = datetime.datetime.now()
        last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # check diff
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            




    # EXECUTE?


    # EXECUTE TRUE
    if EXECUTE == True:
        print("Executtion: ", str(EXECUTE), "... STARTING NOW")

        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        # try to run script, if it fails, it fails
        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "... t'was a SUCCESS! :)"
            print(status)
        except:
            status = "... t'was a FAILURE. :("
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")




    # EXECUTE FALSE
    elif EXECUTE == False:
        print("Executtion: ", str(EXECUTE), "... DID NOT EXECUTE NOR LOG")



    print("Finished: ", notebook_path, "\n\n😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋\n\n")
    pass




########################################################
###################### SCHEDULE #######################
########################################################




# ##### BOLIGRAFICA #####
# SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_covid_replies/covid_reply.ipynb", schedule_in_days= 12/24)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb", schedule_in_days=15/24)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb", schedule_in_days=14/24)

###### GREENHOUSE DATA #####
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb", schedule_in_days = 6/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/random_fig/random_fig.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb", schedule_in_days = 2)


print("🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻\n               FINITO\n🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻")

starting to check:  BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb
previous log found: 2022-09-12 12:36
difference in execution is: 0.9996 days
needed difference to execute: 0.625 days
Executtion:  True ... STARTING NOW
día de la semana hora local UTC = 1
hoy es 2022-09-13
Hora actual en UTC = 12:35:22
envs failed to load

MAPA DEL DÍA:
Paridad de género en años de educación de jóvenes (25-35 años), 2012 

Municipio más álto: Quirusillas con 1.11
Municipio más bajo: Ayata con 0.61

(Fuente: SDSN Bolivia)

TWEET SUCCESFUL!! 
... t'was a SUCCESS! :)
New log entry successful!


Finished:  BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb
previous log found: 2022-09-12 12:36
difference in execution is: 0.9996 days
needed difference to execute: 0.5833 days
Executtion:  True ... STARTING NOW
día de la semana hora loca

Using state Virginia server backend.



Bolivia: Maestros capacitados en educación primaria, mujer (% de maestras) 

Datos vía la API del Banco Mundial

TWEET SUCCESFUL!! 
... t'was a SUCCESS! :)
New log entry successful!


Finished:  BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb
previous log found: 2022-09-12 16:26
difference in execution is: 0.8399 days
needed difference to execute: 1 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb
previous log found: 2022-09-12 20:18
difference in execution is: 0.6788 days
needed difference to execute: 0.25 days
Executtion:  True ... STARTING NOW
envs failed to load
Iran, Islamic Republic of greenhouse gas emission data and fig

['country_data/NOR_Norway/figures/NOR_relative_totals.png',
 'country_data/NOR_Norway/figures/NOR_GCP_Country_Highlight.png',
 'country_data/NOR_Norway/figures/NOR_GCP_1.png',
 'country_data/NOR_Norway/figures/NOR_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True] 



Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Eritrea"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Russian Federation"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Uganda"
 > found 20 tweets
 > days of invalidity: 20
 > Number of invalids: 61
 > 2 usernames filtered out
 > retweets removed: 18


...

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [True, False, False] 



Starting success list [True, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Vietnam"
 > found 2 tweets
 > days of invalidity: 20
 > Number of invalids: 61
 > 2 usernames filtered out
 > retweets removed: 0


['country_data/VNM_Vietnam/figures/VNM_relative_totals.png',
 'country_data/VNM_Vietnam/figures/VNM_GCP_Country_Highlight.png',
 'country_data/VNM_Vietnam/figures/VNM_GCP_1.png',
 'country_data/VNM_Vietnam/figures/VNM_Minx_top20_subsectors.png']

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [True, False, False, False] 



Starting success list [True, False, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Moldova"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Niue"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Afghanistan"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Congo, The Democratic Republic of the"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Greece"
 > found 0 tweets
 > empty df: NEXT ITERATION


['country_data/KOR_South Korea/figures/KOR_relative_totals.png',
 'country_data/KOR_South Korea/figures/KOR_GCP_Country_Highlight.png',
 'country_data/KOR_South Korea/figures/KOR_GCP_1.png',
 'country_data/KOR_South Korea/figures/KOR_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True, False, False, False, True] 





 FINISHED WHILE LOOP


 FINISHED WHILE LOOP
... t'was a SUCCESS! :)
New log entry successful!


Finished:  Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/random_fig/random_fig.ipynb
previous log found: 2022-09-12 16:26
difference in execution is: 0.8403 days
needed difference to execute: 1 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/random_fig/random_fig.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb
previous log found: 2022-09-11 14:14
difference in execution is: 1.9319 days
needed difference to execute: 2 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻
               FINITO
🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻
